In [1]:
# thanks to: 
# https://www.python-course.eu/python3_properties.php
# http://blog.lerner.co.il/python-attributes/
# https://www.programiz.com/python-programming/property

# this node can store any data types including itself.
class Node(object):    
    def __init__(self, dtype=object, max_children=-1):
        self._dtype = dtype
        self.value = dtype() #initialize
        self._max = max_children
        self._children = []    
            
    @property
    def value(self):        
        return self._data
    
    @value.setter
    def value(self, val):        
        if not isinstance(val, self._dtype):
            raise TypeError('Oooops {} data type cannot be stored in this Node({})'.format(type(val), self._dtype))
        self._data = val
    
    def add_child(self, value=None):
        if self._max == -1 or len(self._children) < self._max:
            self._children.append(Node(self._dtype, self._max))
            if value is not None:
                self._children[-1].value = value
            return len(self._children)
        else:
            raise ValueError('This node cannot adopt new child!')
            
    def del_child(self, child_index=-1):        
        assert -1 <= child_index < len(self._children), 'Invalid child index'        
        if len(self._children) == 0:
            return
        del(self._children[child_index])
    
    def get_child(self, child_index):
        assert child_index >= 0, 'Invalid child number'
        if child_index > len(self._children):
            raise ValueError('This node does not have this child')
        return self._children[child_index]
    
    def __getitem__(self, key):
        assert key >=0 and key < len(self._children), 'Invalid child index'
        return self.get_child(key)
    
    def __setitem__(self, key, value):
        assert key >=0 and key < len(self._children), 'Invalid child index'
        assert isinstance(value._dtype, self._dtype), 'dtypes do not match! (%s vs. %s)'\
                                                % (value._dtype, self._dtype)
        assert value._max == self._max, 'Maximum children does not match'
        self._children[key] = value
    
    def count_children(self):
        return len(self._children)
    
    def __repr__(self):
        return '{basetype:%s, child_count:%s, max_children:%s, data:{dtype:%s, value:%s}}' \
                %(self._dtype, len(self._children), self._max, type(self._data), self._data)
        
        
# class Tree(object):
#     def __init__(self, dtype, max_number_of_children):
#         self._dtype = dtype
#         self._root = Node(None, dtype, max_number_of_children)
    
#     def root(self):
#         return(self._root)

In [2]:
n = Node(object,2)
n.value = 'root'
n.add_child('l')
n.add_child('r')
n[0].add_child('ll')
n[0].add_child('lr')
n[1].add_child('rl')
print(n)
print(n[0])
print(n[1])
print(n[0][0])
print(n[0][1])
print(n[1][0])

{basetype:<class 'object'>, child_count:2, max_children:2, data:{dtype:<class 'str'>, value:root}}
{basetype:<class 'object'>, child_count:2, max_children:2, data:{dtype:<class 'str'>, value:l}}
{basetype:<class 'object'>, child_count:1, max_children:2, data:{dtype:<class 'str'>, value:r}}
{basetype:<class 'object'>, child_count:0, max_children:2, data:{dtype:<class 'str'>, value:ll}}
{basetype:<class 'object'>, child_count:0, max_children:2, data:{dtype:<class 'str'>, value:lr}}
{basetype:<class 'object'>, child_count:0, max_children:2, data:{dtype:<class 'str'>, value:rl}}


In [3]:
node = Node(Node, 3)
print(node.value)
node.value = Node(dict)
print(node.value)
print(node.value.value)

{basetype:<class 'object'>, child_count:0, max_children:-1, data:{dtype:<class 'object'>, value:<object object at 0x7f6cf9feabb0>}}
{basetype:<class 'dict'>, child_count:0, max_children:-1, data:{dtype:<class 'dict'>, value:{}}}
{}


In [ ]:
class LinkedList(object):
    class ListNode(object):
        def __init__(self, data):
            self._data = data
            self.next = None
            
    def __init__(self, data):
        self.head = ListNode(data)
        self.head.next = None

In [ ]:
t = Tree(2)

In [ ]:
r = t.root()

In [ ]:
print(r.value(5))

In [ ]:
x = int

# Tips

## Decorators

In [4]:
# Thanks to https://www.programiz.com/python-programming/decorator

def star(func):
    def inner(*args, **kwargs):
        print('*' * 30)
        func(*args, **kwargs)
        print('*' * 30)
        return
    return inner

def nonnegative_integer(func):
    def inner(x):
        if not isinstance(x, int):
            raise TypeError('factorial is not defined for {}'.format(x))
        if x < 0:
            raise ValueError('factorial is not defined for negative numbers')
        return func(x)
    return inner

def printer(func):
    def inner(*args, **kwargs):
        print(func(*args, **kwargs))
    return inner
        

@nonnegative_integer
def factorial(n:int):
    if n==0 or n==1:
        return 1
    return n * factorial(n-1)

@star
@printer
@nonnegative_integer
def fact(n:int):
    if n==0 or n==1:
        return 1
    res=1
    for i in range(1,n+1):
        res *= i
    return res


def f(n:int):
    if n==0 or n==1:
        return 1
    res=1
    for i in range(1,n+1):
        res *= i
    return res
nondecorated_fact = star(printer(nonnegative_integer(f)))

print(factorial(5))
fact(5)
nondecorated_fact(5)

120
******************************
120
******************************
******************************
120
******************************
